In [1]:
import os,sys
import time
import numpy as np
import pandas as pd

In [2]:
from sklearn.feature_selection import SelectKBest, SelectPercentile, SelectFdr, SelectFpr, SelectFwe
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE,RFECV
from sklearn.feature_selection import chi2,f_classif, mutual_info_classif
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.linear_model import LogisticRegression,LassoCV, Lasso
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier, VotingClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA, NMF
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB
from xgboost import XGBClassifier
from tpot import TPOTClassifier

In [3]:
from preprocessing.Normalize import Normalize
import helper.SeriesHelper as series_helper

In [4]:
normal_matrix = Normalize().get_normalized_data()
X = normal_matrix.to_numpy()
y = series_helper.get_relapse_value_from_series_matrix(normal_matrix)

In [5]:
cols = list(normal_matrix.columns)

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33)

In [28]:
n_jobs = 6
verbose =  1

In [29]:
m1 = Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=n_jobs,
                              transformer_list=[('pca',
                                                 PCA(copy=True,
                                                     iterated_power='auto',
                                                     n_components=100,
                                                     random_state=None,
                                                     svd_solver='auto', tol=0.0,
                                                     whiten=False)),
                                                ('univ_select',
                                                 SelectKBest(k=500,
                                                             ))],
                              transformer_weights=None, verbose=verbose)),
                ('svm',
                 SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='linear', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)


In [30]:
m2 = Pipeline(memory=None,
         steps=[('feature_selection',
                 NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0,
                     max_iter=200, n_components=100, random_state=None,
                     shuffle=False, solver='cd', tol=0.0001, verbose=0)),
                ('model',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=n_jobs,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=verbose)

In [31]:
m3 = Pipeline(memory=None,
         steps=[('feature_selection',
                 NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0,
                     max_iter=200, n_components=100, random_state=None,
                     shuffle=False, solver='cd', tol=0.0001, verbose=0)),
                ('model',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=n_jobs,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=verbose)

In [32]:
m4 = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=n_jobs,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=verbose)

In [33]:
m5 = Pipeline(memory=None,
         steps=[('feature_selection',
                 NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0,
                     max_iter=300, n_components=100, random_state=None,
                     shuffle=False, solver='cd', tol=0.0001, verbose=0)),
                     ('model',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=5000,
                                    multi_class='auto', n_jobs=n_jobs,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=verbose)

In [34]:
m6 = Pipeline(memory=None,
         steps=[('feature_selection',
                 NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0,
                     max_iter=400, n_components=100, random_state=None,
                     shuffle=False, solver='cd', tol=0.0001, verbose=0)),
                     ('model',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=5000,
                                    multi_class='auto', n_jobs=n_jobs,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],verbose=verbose)

In [39]:
voting_hard = VotingClassifier(estimators=[
    ('m1',m1),('m2',m2),('m3',m3),('m4',m4),('m5',m5),('m6',m6)
],voting='hard',n_jobs=n_jobs)

TypeError: __init__() got an unexpected keyword argument 'verbose'

In [36]:
from sklearn.metrics import precision_score

In [37]:
for x in [m1,m2,m3,m4,m5,m6]:
    t = time.time()    
    print("-------------"*8)
    print("Fitting Started")
    x.fit(X_train,y_train)
    print("Fitting Ended")
    print(" Train Score : ",x.score(X_train,y_train),"\t Test Score ",x.score(X_test,y_test))
    print(" Precision Train : ",precision_score(y_train,x.predict(X_train))," Precision Test : ",precision_score(y_test,x.predict(X_test)))
    print("Time Required : ",time.time()-t)

--------------------------------------------------------------------------------------------------------
Fitting Started
Fitting Ended
 Train Score :  0.8252184769038702 	 Test Score  0.6590909090909091
 Precision Train :  0.9142857142857143  Precision Test :  0.5679012345679012
Time Required :  4.534290075302124
--------------------------------------------------------------------------------------------------------
Fitting Started
[Pipeline] . (step 1 of 2) Processing feature_selection, total= 1.2min
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.1s
Fitting Ended
 Train Score :  0.7265917602996255 	 Test Score  0.6565656565656566
 Precision Train :  0.722972972972973  Precision Test :  0.6041666666666666
Time Required :  113.17234492301941
--------------------------------------------------------------------------------------------------------
Fitting Started
[Pipeline] . (step 1 of 2) Processing feature_selection, total= 1.1min
[Pipeline] ............. (step 2 of 

In [40]:
voting_hard.fit(X_train,y_train)

VotingClassifier(estimators=[('m1',
                              Pipeline(memory=None,
                                       steps=[('features',
                                               FeatureUnion(n_jobs=6,
                                                            transformer_list=[('pca',
                                                                               PCA(copy=True,
                                                                                   iterated_power='auto',
                                                                                   n_components=100,
                                                                                   random_state=None,
                                                                                   svd_solver='auto',
                                                                                   tol=0.0,
                                                                                   whiten=False)),
 

In [42]:
voting_hard.score(X_test,y_test)

0.6590909090909091

In [70]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = cols
    # calculate the correlation with y for each feature
    for i in cols:
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(normal_matrix, y,2000)
print(str(len(cor_feature)), 'selected features')

2000 selected features


In [71]:
X_train,X_test,y_train,y_test = train_test_split(normal_matrix[cor_feature],y,test_size=0.33)

In [9]:
nmf = NMF(n_components=400).fit(X,y)

In [73]:
temp_train = nmf.transform(X_train)
temp_test = nmf.transform(X_test)

In [74]:
lr = LogisticRegression(max_iter=10000)

In [75]:
lr.fit(temp_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [76]:
lr.score(temp_train,y_train)

1.0

In [77]:
lr.score(temp_test,y_test)

0.803030303030303

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(11800, input_dim=11800, activation='relu'))
    model.add(Dense(5600, activation='relu'))
    # model.add(Dense(3000, activation='relu'))
    # model.add(Dense(1500, activation='relu'))
    # model.add(Dense(750, activation='relu'))
    model.add(Dense(300, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','binary_accuracy',])
    return model

In [20]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X,y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))